In [1]:
import torch
import torchtext
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import pandas as pd
import re
from collections import Counter
from tqdm.notebook import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 能用gpu则用gpu

# 1.准备数据

In [2]:
class MyDataset(Dataset):
    def __init__(self, file_path, tokenizer, stopwords, sample=None):
        df = pd.read_csv(file_path)
        df = df.dropna().reset_index(drop=True)
        if sample:
            df = df.sample(sample).reset_index(drop=True)
            
        counter = Counter()
        sentences = []
        for title in tqdm(df['title']):
            # 去除标点符号
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
            tokens = [token for token in tokenizer(title.strip()) if token not in stopwords]
            counter.update(tokens)
            sentences.append(tokens)
            
        self.vocab = torchtext.vocab.vocab(counter, specials=['<unk>', '<pad>'])
        self.vocab.set_default_index(self.vocab['<unk>'])

        self.inputs = [self.vocab.lookup_indices(tokens) for tokens in sentences]
        self.labels = [[label] for label in df['label'].values.tolist()]
        self.n_class = len(df['label'].unique())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])

In [4]:
file_path = '../../datasets/THUCNews/train.csv'
tokenizer = torchtext.data.utils.get_tokenizer('spacy', language='zh_core_web_sm')
stopwords = [line.strip() for line in open('../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]
dataset = MyDataset(file_path, tokenizer, stopwords, sample=10000)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [5]:
def collate_fn(batch_data):
    return pad_sequence([x for x, y in batch_data], padding_value=1), torch.tensor([y for x, y in batch_data]).unsqueeze(1)

dataloader = DataLoader(dataset, batch_size=1024, shuffle=True, collate_fn=collate_fn)

# 2.构建模型

In [15]:
class TextRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, n_class, n_hidden):
        super().__init__()
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # rnn层，深度为1
#         self.rnn = nn.RNN(embed_size, n_hidden, 1, batch_first=True)
#         self.rnn = nn.LSTM(embed_size, n_hidden, 1, batch_first=True)  # LSTM
        self.rnn = nn.LSTM(embed_size, n_hidden, 2, batch_first=True)  # 双向LSTM
#         self.rnn = nn.GRU(embed_size, n_hidden, 1, batch_first=True)  # GRU
#         self.rnn = nn.GRU(embed_size, n_hidden, 2, batch_first=True)  # 双向GRU
        
        # 激活函数
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        # 输出头
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, x):  # x: [batch_size * 句子长度]
        x = x.permute(1, 0)
        x = self.embedding(x)  # [batch_size * 句子长度 * embed_size]
        out, _ = self.rnn(x)  # [batch_size * 句子长度 * n_hidden]
        out = self.relu(out)
        out = self.dropout(out)
        logits = self.fc(out[:, -1, :])  # 全连接输出头，[batch_size * n_class]
        return logits

In [16]:
model = TextRNN(vocab_size=len(dataset.vocab), embed_size=256,
                n_class=dataset.n_class, n_hidden=256).to(device)
model

TextRNN(
  (embedding): Embedding(26106, 256)
  (rnn): LSTM(256, 256, num_layers=2, batch_first=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=256, out_features=14, bias=True)
)

# 3.训练模型

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().to(device)
model.train()
for epoch in range(200):
    for feature, target in dataloader:
        feature = feature.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        logits = model(feature)
        loss = criterion(logits, target.squeeze())
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, ', loss:', format(loss.item(), '.6f'))

epoch: 1 , loss: 2.292954
epoch: 2 , loss: 2.231810
epoch: 3 , loss: 2.153455
epoch: 4 , loss: 2.069539
epoch: 5 , loss: 1.944275
epoch: 6 , loss: 1.893471
epoch: 7 , loss: 1.715781
epoch: 8 , loss: 1.512021
epoch: 9 , loss: 1.442197
epoch: 10 , loss: 1.228781
epoch: 11 , loss: 1.154802
epoch: 12 , loss: 1.150409
epoch: 13 , loss: 1.082944
epoch: 14 , loss: 1.122523
epoch: 15 , loss: 1.064640
epoch: 16 , loss: 0.954334
epoch: 17 , loss: 0.879725
epoch: 18 , loss: 0.808169
epoch: 19 , loss: 0.713263
epoch: 20 , loss: 0.682258
epoch: 21 , loss: 0.627411
epoch: 22 , loss: 0.473250
epoch: 23 , loss: 0.434862
epoch: 24 , loss: 0.331124
epoch: 25 , loss: 0.306136
epoch: 26 , loss: 0.284353
epoch: 27 , loss: 0.279979
epoch: 28 , loss: 0.184840
epoch: 29 , loss: 0.146139
epoch: 30 , loss: 0.191492
epoch: 31 , loss: 0.070637
epoch: 32 , loss: 0.087986
epoch: 33 , loss: 0.082556
epoch: 34 , loss: 0.045362
epoch: 35 , loss: 0.096893
epoch: 36 , loss: 0.106768
epoch: 37 , loss: 0.108829
epoch: 38 

# 4.预测

In [18]:
model.eval()
df_train = pd.read_csv('../../datasets/THUCNews/train.csv')
df_test = pd.read_csv('../../datasets/THUCNews/test.csv')

In [20]:
for i, row in df_test.sample(10).iterrows():
    title = row['title']
    actual = row['class']
    title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
    tokens = [token for token in tokenizer(title.strip()) if token not in stopwords]
    inputs = dataset.vocab.lookup_indices(tokens)
    inputs = torch.LongTensor(inputs).unsqueeze(1).to(device)
    predict = model(inputs)
    predict_class = dict(zip(df_train['label'], df_train['class']))[predict.max(1)[1].item()]    
    print(''.join(tokens), '||| actual:', actual, ', predict:', predict_class)

前裁判因未领会领导暗示提前退役庆幸离开圈子 ||| actual: 体育 , predict: 体育
市场份额降至升至图 ||| actual: 科技 , predict: 股票
费根停摆危害显现临时工失业休城商业受创 ||| actual: 体育 , predict: 娱乐
精简裁员传诺基亚已经取消 ||| actual: 科技 , predict: 股票
美国参议院批准奥巴马政府首批成员名单 ||| actual: 时政 , predict: 股票
外放效果一流五款专业音乐手机全选择 ||| actual: 科技 , predict: 教育
英超荷兰杀手梅开二度利物浦近战胜逼曼联 ||| actual: 体育 , predict: 体育
家用电器关注洗衣机消费升级 ||| actual: 股票 , predict: 科技
售价合理功能出众元品牌手机推荐 ||| actual: 科技 , predict: 科技
球衣销量排行科比湖双登顶火箭上榜 ||| actual: 体育 , predict: 娱乐
